In [31]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [32]:
# Loasd the trained model, scalar pickel, onhot
model=load_model("model.h5")

# Load the encoder and scalar
with open("LE.pkl",'rb') as file:
    lable_encoder_gender=pickle.load(file)
with open("one_hot_encoder.pkl","rb") as file:
    lable_encoder_geo=pickle.load(file)
with open("scaler.pkl","rb") as file:
    scalar=pickle.load(file)
    

In [33]:
# Example input data
input_data={
    "CreditScore":600,
    "Geography":"France",
    "Gender":"Male",
    "Age":40,
    "Tenure":3,
    "Balance":6000,
    "NumOfProducts":2,
    "HasCrCard":1,
    "IsActiveMember":1,
    "EstimatedSalary":50000
}

In [30]:
geo_encoded=lable_encoder_geo.transform([[input_data["Geography"]]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=lable_encoder_geo.get_features_names_out(["Geography"]))
geo_encoded_df

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


AttributeError: 'OneHotEncoder' object has no attribute 'get_features_names_out'

In [34]:
geo_encoded = lable_encoder_geo.transform([[input_data["Geography"]]]).toarray()

if hasattr(lable_encoder_geo, "get_feature_names_out"):
    cols = lable_encoder_geo.get_feature_names_out(["Geography"])
else:
    cols = lable_encoder_geo.get_feature_names(["Geography"])

geo_encoded_df = pd.DataFrame(geo_encoded, columns=cols)
geo_encoded_df

c:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [35]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,6000,2,1,1,50000


In [36]:
# Encode cat var
input_df["Gender"]=lable_encoder_gender.transform(input_df["Gender"])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,6000,2,1,1,50000


In [37]:
# concinate one hot encoded
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encoded_df],axis=1)
input_df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,6000,2,1,1,50000,1.0,0.0,0.0


In [38]:
# Scalling the data
input_scaled=scalar.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -1.12240462,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [39]:
# Pridictionn churn
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 450ms/step


array([[0.02418464]], dtype=float32)

In [40]:
prediction_proba=prediction[0][0]
prediction_proba

0.024184637